### A notebook to test parallelization of the `Ais` class in `NPS-ActiveSpace.utils.models`

In [ ]:
import datetime as dt
import cmocean as cm
import glob
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from shapely.geometry import Point
from scipy import stats
import sys
from tqdm import tqdm

# general NSNSD acoustical tools
# =======================================================================================================
repo_dir = r"C:\Users\DBetchkal\PythonScripts\GITHUB" # ADJUST TO YOUR LOCAL DIRECTORY
# =======================================================================================================
sys.path.append(os.path.join(repo_dir, "iyore"))
sys.path.append(os.path.join(repo_dir, "NPS-ActiveSpace"))

import iyore
import nps_active_space
from nps_active_space.utils.models import Ais

def NVSPL_to_AIS_subset(acoustic_ds, vessel_ds, unit, site):
    """
    Use the dates of available acoustic data (as NVSPL files)
    to reference the paths of corresponding transportation data (as raw AIS files).

    This allows the analyst to parse only the relevant data.
    """
    
    AIS_to_parse = []
    for entry in tqdm(acoustic_ds.nvspl(unit=unit, site=site), unit="NVSPL files"):
        paths = [e.path for e in vessel_ds.AIS(year=int(entry.year), month=int(entry.month), day=int(entry.day))] # WILL BE OFF BY +9 HOURS! 😧
        for path in paths:
            AIS_to_parse.append(path) 
    AIS_to_parse = np.unique(AIS_to_parse)

    return AIS_to_parse

# import pytz
# set(pytz.all_timezones_set)

### Step 1: compile (raw) AIS data filepaths into a list

In [ ]:
# where are the AIS data stored? (with the requisite `iyore` .structure.txt file)
# csv_path = r"C:\Users\DBetchkal\Documents\ArcGIS\Projects\GLBA_AIS_2022\MXAK-AIS-GLBA"
csv_path = r"V:\Noncanonical Data\2024 - 2020 MXAK-AIS-GLBA\MXAK-AIS-GLBA"
ds_AIS = iyore.Dataset(csv_path) # initialize an `iyore` dataset object

ds = iyore.Dataset(r"E:\Sound Data") # where are the acoustic data stored?
# iyore_subset = NVSPL_to_AIS_subset(ds, ds_AIS, "GLBA", "RENDU") # this function finds every AIS file which corresponds to an acoustic record (a bit slow)
# print("OK, we found", len(iyore_subset), "relevant AIS dates to parse")

# or you can just work with every single AIS file
# iyore_subset = glob.glob(r"C:\Users\DBetchkal\Documents\ArcGIS\Projects\GLBA_AIS_2022\MXAK-AIS-GLBA\MXAK-AIS-GLBA-*.csv")
iyore_subset = glob.glob(csv_path+os.sep+"*.csv")

iyore_subset

### Step 2: parse the entire AIS dataset
#### Without parallelization the parsing rate was 4.7 seconds per file (02:04:10 to load 1585 files)
<font color="green">Success! with parallelization </font> (over 32 logical processors on \\inpdenagrogu), the rate has improved to (00:19:40 to load 1585 files) or *1.34 seconds per file* <br>

In [ ]:
start_time = dt.datetime.now() # we'd like to keep track of the overall processing time required
AIS_gdf = Ais(list(iyore_subset)) # we load the AIS record in parallel
end_time = dt.datetime.now()

print(f"Processed {len(list(iyore_subset))} AIS files in {end_time - start_time}, {((end_time - start_time)/len(list(iyore_subset))).total_seconds():.2f} seconds/file")

# bay_AIS = AIS_gdf.cx[-137.134247:-135.747100, 58.426598: 59.105969].copy() # a rectangular bounding box containing Glacier Bay proper

In [ ]:
bay_AIS = AIS_gdf.cx[-137.134247:-135.747100, 58.426598: 59.105969].copy() # a rectangular bounding box containing Glacier Bay proper

In [ ]:
bay_AIS.loc[bay_AIS["MMSI"] == 367365630, :].plot(color="k", markersize=1, alpha=0.01)
plt.show()

In [ ]:
# events = pd.Series()
# for mmsi_, pts in bay_AIS.groupby('MMSI'):
#     events[mmsi_] = len(pts['event_id'].unique())

events.sort_values(ascending=False).head(200).to_csv(r"C:\Users\DBetchkal\Desktop\common vessels.csv")

### Step 3: find vessels passing nearby certain potential array locations
if this cell throws an error, try running it a second time?

In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
gdf = gpd.read_file(r"T:\ResMgmt\WAGS\Sound\ArrayScoping2025.kmz", driver="LIBKML")
gdf.boundary.plot()
plt.show()

# gdf_out = gdf.to_crs("EPSG:4326")
# gdf_out.to_file(r"T:\ResMgmt\WAGS\Sound\GLBA Array Scoping 2025.geojson", driver="GeoJSON")

In [ ]:
names = ["North Marble W", "North Marble E", "Willoughby NW", "Willoughby SE"]
out = gpd.GeoDataFrame([])

for array_bound, name in zip(gdf.geometry.explode(index_parts=True)[0], names):

    array_pot=gpd.GeoSeries(array_bound, crs="EPSG:4326")
    array_pot_ea = array_pot.to_crs("EPSG:26908")
    sample_area = (array_pot_ea.area[0]/1e6) # km^2
    
    print(f"Now computing intersection with {name} array.")
    AIS_clipped = gpd.clip(gdf=AIS_gdf, mask=array_pot)
    AIS_clipped = AIS_clipped.sort_values('heading') # we want it organized by heading
    AIS_ = AIS_clipped.loc[((AIS_clipped["TIME"].dt.month >= 5)&(AIS_clipped["TIME"].dt.month < 10)), :] # select only summertime events
    AIS_.to_file(os.path.join(r"C:\Users\DBetchkal\Desktop", name+" clipped points.geojson"), driver='GeoJSON')
    print(f"\tClipped to an area of {sample_area:.1f} km^2. Visualizing...")

    fig,ax = plt.subplots(1,1, figsize=(8,7))
    array_pot.to_crs("EPSG:26908").boundary.plot(ax=ax, color="k", ls="--")
    AIS_.to_crs("EPSG:26908").plot('heading', cmap=cmocean.cm.phase, markersize=1, alpha=0.06, ax=ax, 
                                   legend=True, legend_kwds={"shrink":.5, "ticks":[0,90,180,270,359], "label":"Heading (°)", "aspect":10})
    ax.set_title(f"{name}:  {len(AIS_.MMSI.unique()) :.0f} unique vessels by MMSI", loc="left", y=1.03)
    ax.set_xlabel("Easting (m)", labelpad=20)
    ax.set_ylabel("Northing (m)", labelpad=20)
    ax.ticklabel_format(useOffset=False, style='plain')
    plt.savefig(os.path.join(r"C:\Users\DBetchkal\Desktop", name+" array scoping map.png"), dpi=300, bbox_inches="tight", facecolor="white")
    plt.show()
    plt.close()

    print(f"\tPoints have been selected.")

    S = stats.entropy(AIS_.MMSI)
    out.loc[name, "unique_MMSI"] = len(AIS_["MMSI"].unique())
    out.loc[name, "S"] = S # entropy, in bits
    
    print(f"\tVessel entropy = {S:.1f} bits.")
    
out